## Data Cleaning
### Context
The dataset is comprised of information about songs (tracks), music genres, albums, and artists as aggregated from the Free Music Archive (FMA). In particular, this data was used in an academic study on music information retrieval; both the paper and data is accessible from the following [github repository](https://github.com/mdeff/fma). The dataset itself contains 917 GB and 343 days of Creative Commons-licensed audio from 106,574 tracks from 16,341 artists and 14,854 albums, arranged in a hierarchical taxonomy of 161 genres. For the purposes of this project, we are only concerned with a subset of this data that actually has documentation of specific measurable musical characteristics such as tempo, energy, and danceability, to name a few. 

### Cleaning
We go through various steps to clean the data across the several files provided from the FMA. Much of the cleaning and data manipulation pertains to text editing and ensuring data is consistent across dataframes which were then loaded into a relation database. 

In [1]:
# load necessary libraries
import os
import sqlalchemy
import numpy as np
import pandas as pd

from dotenv import load_dotenv


# load variables from .env file into set of environment variables
load_dotenv()

# get environment variables for database connection
hostname = os.getenv("HOSTNAME")
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
database = os.getenv("DATABASE")
port = 3306

# create db connection str and MySQL engine pool
connect_str = f"mysql+pymysql://{username}:{password}@{hostname}:3306/{database}"
engine = sqlalchemy.create_engine(connect_str)

In [ ]:
# load song/track characteristics data
echo_df = pd.read_csv("../Data/fma_metadata_updated/echonest_updated.csv")

# only keep desired columns
echo_df = echo_df.iloc[:, :9]

In [ ]:
# load data for songs/tracks
tracks_df = pd.read_csv("../Data/fma_metadata_updated/raw_tracks.csv")

# filter and only keep desired columns
tracks_df = tracks_df.filter([
    "track_id",
    "album_id",
    "artist_id",
    "track_duration",
    "track_explicit",
    "track_favorites",
    "track_listens",
    "track_title",
    "track_genres"
])

# inner join tracks with characteristics; only keep songs with characteristic information
tracks_echo_df = pd.merge(
    left=echo_df,
    right=tracks_df,
    how="inner",
    on="track_id"
)

# rename columns
tracks_echo_df.rename(
    {
        "track_duration": "duration",
        "track_explicit": "explicit",
        "track_favorites": "favorites",
        "track_listens": "listens",
        "track_title": "title",
        "track_genres": "genre_id"
    }, 
    axis=1, 
    inplace=True
)

# some preliminary data cleaning and data type conversions
tracks_echo_df["duration"] = tracks_echo_df.duration.apply(lambda x: int(x.split(":")[0])*60 + int(x.split(":")[1])) # convert song duration from min:sec format to seconds
tracks_echo_df["album_id"] = tracks_echo_df.album_id.apply(int) # convert to int datatype
tracks_echo_df["explicit"] = tracks_echo_df["explicit"].replace({"Radio-Unsafe": 1, "Radio-Safe": 0, np.nan: -1}) # replace with levels 1 being explicit, 0 be not explicit, and -1 being unsure
tracks_echo_df["genre_id"] = tracks_echo_df.genre_id.apply(lambda x: [genre["genre_id"] for genre in eval(x)]) # extract genre_id from list of genre dicts

In [4]:
# load data for genres
genres_df = pd.read_csv("../Data/fma_metadata_updated/genres_updated.csv")

# map tracks with multiple genres to individual rows
tracks_genres_df = tracks_echo_df.explode("genre_id")
tracks_genres_df["genre_id"] = tracks_genres_df["genre_id"].astype(int)

# match songs with only top-level genres (16 unique genres), eliminating niche genres (160+ total)
tracks_genres_df = pd.merge(
    left=pd.merge(                  # join genres with their top-level genre in the hiearchy
        left=genres_df,
        right=genres_df,
        how="inner",
        left_on="genre_id",
        right_on="top_level"
    ).iloc[:, :7],
    right=tracks_genres_df[["track_id", "genre_id"]],
    how="inner",
    left_on="genre_id_y",
    right_on="genre_id"
)

# drop unnecessary columns and rename others
tracks_genres_df = tracks_genres_df.drop(["genre_id_y","genre_id", "top_level_x", "parent_x"], axis=1)
tracks_genres_df = tracks_genres_df.drop_duplicates()
tracks_genres_df.rename(
    {
        "genre_id_x":"genre_id", 
        "#tracks_x":"num_tracks",
        "title_x": "title",
        "genre_color_x": "genre_color",
    },
    axis=1, 
    inplace=True
)

# create dataframe of genres
genres_df = tracks_genres_df.filter(["genre_id", "num_tracks", "title", "genre_color"]).drop_duplicates().reset_index(drop=True)

# create dataframe mapping tracks to top-level genres
tracks_genres_df = tracks_genres_df.filter(["track_id", "genre_id"]).drop_duplicates().reset_index(drop=True)

# create tracks dataframe, dropping a few erraneous observations
tracks_df = tracks_echo_df.drop("genre_id", axis=1)
tracks_df = tracks_df[~tracks_df.artist_id.isin([2442, 13401])]
tracks_df.loc[tracks_df.artist_id == 1680,"artist_id"] = 4542

In [ ]:
# load artist data
artists_df = pd.read_csv("../Data/fma_metadata_updated/raw_artists.csv")

# filter and only keep desired columns
artists_df = artists_df.filter(["artist_id","artist_favorites", "artist_name"])

# only keep artists that have made atleast one track
artists_df = artists_df[artists_df["artist_id"].isin(tracks_df.artist_id)]

# rename columns
artists_df.rename({"artist_favorites":"favorites", "artist_name":"name"}, axis=1, inplace=True)
artists_df.reset_index(drop=True, inplace=True)

# drop a few erraneous observations
artists_df = artists_df[~artists_df.artist_id.isin([2442, 13401])]
artists_df.loc[artists_df.artist_id == 1680, "artist_id"] = 4542

In [9]:
# load album data
albums_df = pd.read_csv("../Data/fma_metadata_updated/raw_albums.csv")

# only keep albums which have a matching artist name
albums_df = albums_df[albums_df["artist_name"].isin(artists_df.name)]

# swap matching artist names with artist ids to comply with sql schema
# swap self-titled albums (s/t) with the artist name
albums_df = pd.merge(
    left=albums_df,
    right=artists_df[["name", "artist_id"]],
    how="inner",
    left_on="artist_name",
    right_on="name"
)
albums_df["album_title"] = albums_df.apply(lambda row: row["artist_name"] + "(s/t)" if row["album_title"].lower() == "s/t" else row["album_title"], axis=1)

# filter and only keep desired columns
albums_df = albums_df.filter([
    "album_id", 
    "album_date_released", 
    "album_favorites", 
    "album_title", 
    "album_tracks", 
    "album_listens", 
    "artist_id"
])

# rename columns
albums_df.rename({ 
    "album_date_released": "release_date", 
    "album_favorites": "favorites", 
    "album_title": "title", 
    "album_tracks": "num_tracks", 
    "album_listens": "listens", 
    },
    axis=1,
    inplace=True
)

# convert release date to datetime format
albums_df["release_date"] = pd.to_datetime(albums_df.release_date, format="%m/%d/%Y")
albums_df.reset_index(drop=True, inplace=True)

# show head
albums_df.head()

,album_id,release_date,favorites,title,num_tracks,listens,artist_id
0,1,2009-01-05,4,AWOL - A Way Of Life,7,6073,1
1,13756,2012-04-01,0,Jackin4Beats,11,4995,1
2,100,2009-01-09,0,On Opaque Things,4,5613,80
3,101,2007-01-01,0,Wooden Lake Sexual Diner,3,2180,80
4,10308,2011-10-03,0,"Live at WFMU on Liz Berg's Show on October 3, ...",9,3163,80


In [10]:
# remap id in each dataframe to coincide with sql table schemas
tracks_df.rename({"track_id": "id"},axis=1, inplace=True)
genres_df.rename({"genre_id": "id"},axis=1, inplace=True)
artists_df.rename({"artist_id": "id"},axis=1, inplace=True)
albums_df.rename({"album_id": "id"},axis=1, inplace=True)

In [11]:
### final cleanup based of dataframes

# string cleanup of artist names a nd track titles
artists_df["name"] = artists_df.name.apply(lambda x: x.split(",")[0].replace("&amp;", "&").replace("&amp", "&").replace("(F.A.G.G)", "").replace("(Various)", "").replace("&;", "&").replace("(avec logo panthère)", "").strip())

# string cleanup of album names
albums_df["title"] = albums_df.title.apply(lambda x: x.strip())

# only keep tracks that have an associate artist
tracks_df = tracks_df[tracks_df.artist_id.isin(artists_df.id)]

# if a track's album id isn't contained in the album data set it to NA
tracks_df.loc[~tracks_df.album_id.isin(albums_df.id), "album_id"] = np.nan

# fix this track title as it is titled NaN and should not be treated as a missing value
tracks_df.loc[tracks_df.title.isna(), "title"] = 'NaN'
tracks_df["title"] = tracks_df.title.apply(lambda x: x.replace("&amp;", "&").replace("&amp", "&"))

# only keep tuples which have a track_id that's still present in the tracks dataframe
tracks_genres_df = tracks_genres_df[tracks_genres_df.track_id.isin(tracks_df.id)]

# drop all albums without a track in the dataset and correct all track counts
albums_df = pd.merge(
    left=tracks_df[["id", "album_id"]].groupby(by="album_id", as_index=False).count(),
    right=albums_df,
    left_on="album_id",
    right_on="id",
    how="inner",
    suffixes=("_counts", None)
).drop(["album_id", "num_tracks"], axis=1).rename({"id_counts": "num_tracks"},axis=1)

In [ ]:
# map sql table to pandas dataframes
tables = dict(
    Genres=genres_df, 
    Artists=artists_df,
    Albums=albums_df, 
    Tracks=tracks_df, 
    Track_Genres=tracks_genres_df,
)

# insert pandas tables into relevant MySQL tables
for table, df in tables.items():
    insertions = df.to_sql(
        table,
        engine,
        if_exists="append",
        index=False,
        chunksize=1000
    )
    print(f"Pandas dataframe inserted into DB {table} table successfully with {insertions} tuples.")

In [ ]:
# check that data was inserted correctly
with engine.connect() as connection:
    for table in tables.keys():
        print(f"{table}\n{'-'*len(table)}")
        query = sqlalchemy.text(f"SELECT * FROM {table} ORDER BY {'id' if table!='Track_Genres' else 'track_id'} ASC LIMIT 10")
        response = connection.execute(query)
        for row in response:
            print(row)
        print()

# dispose of db connection pool and close all connections
engine.dispose()